In [1]:
from ortools.sat.python import cp_model

In [2]:
days = range(5)
day_names = ["Mon","Tues","Wed","Thurs","Fri"]
sessions = range(17)
sess_times = ["08:00 - 10:00","08:30 - 10:30","09:00 - 11:00",
             "09:30 - 11:30","10:00 - 12:00","10:30 - 12:30",
             "11:00 - 13:00","11:30 - 13:30","12:00 - 14:00",
             "12:30 - 14:30","13:00 - 15:00","13:30 - 15:30",
             "14:00 - 16:00","14:30 - 16:30","15:00 - 17:00",
             "15:30 - 17:30","16:00 - 18:00"]
tracers = range(4)
tracers_names = ["P","A","M","U"] #PIB, AV1451, MK6240, UCB-J

In [24]:
first = [1,1,1,1,1,
         1,1,1,1,1,
         1,1,1,1,1,
         1,1]

second = [first, first, first, first, first]
third = [second, second, second, second]
print(third)

[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [3]:
# find all the solutions

# Creates the model.
model = cp_model.CpModel()

In [4]:
#Add new variable for each combination of days, sessions, tracers
#Each day,session,tracer is [0,1] Positive Integers
x = {}
for t in tracers:
    for d in days:
        for s in sessions:
            x[(t,d,s)] = model.NewBoolVar(
                "{}{}{}".format(tracers_names[t],
                                day_names[d],
                                sess_times[s]))
            
            #x[(t,d,s)] = model.NewBoolVar('shift_n%id%is%i' % (t, d, s))
            #x[(t,d,s)] = model.NewBoolVar("x[{},{},{}]".format(d,s,t))
#x = {}
#for s in sessions:
#    for d in days:
#        for t in tracers:
#            x[d,s,t] = model.NewBoolVar("x[{},{},{}]".format(d,s,t))
# old attempt            model.NewIntVar(0,1,"x{}{}{}".format(d,s,t))

In [5]:
# Creates the constraints.
#Each session gets at most 1 tracer
for d in days:
    for s in sessions:
        model.Add(sum(x[(t,d,s)] for t in tracers) <= 1)

In [6]:
#Each tracer gets at least 1 session per week
for t in tracers:
    all_sess = []
    for d in days:
        for s in sessions:
            all_sess.append(x[(t,d,s)])
    model.Add(sum(all_sess) >= 1)

In [7]:
#PIB 3 max per day/batch
t = tracers_names.index("P")
for d in days:
    p_day = []
    for s in sessions:
        p_day.append(x[(t,d,s)])
        model.Add(sum(p_day) <= 3)

In [8]:
#12 per month for AV1451; 3 per week
t = tracers_names.index("A")
a_week = []
for d in days:
    for s in sessions:
        a_week.append(x[(t,d,s)])
model.Add(sum(a_week) <= 3)

In [9]:
#MK6240 max 3 sessions each day
t = tracers_names.index("M")
for d in days:
    m_3 = []
    for s in sessions:
        m_3.append(x[(t,d,s)])
    model.Add(sum(m_3) <= 3)

In [10]:
#UCB-J max 1 session each day
t = tracers_names.index("U")
for d in days:
    u_1 = []
    for s in sessions:
        u_1.append(x[(t,d,s)])
    model.Add(sum(u_1) <= 1)

In [11]:
#2 hr slots without overlap
for d in days:
    for s in sessions[0:14]:
        overlap = []
        for t in tracers:
            overlap.extend([
                x[(t,d,s)],x[(t,d,s+1)],
                x[(t,d,s+2)],x[(t,d,s+3)]
            ])
        model.Add(sum(overlap) <= 1)

In [12]:
#24 HR gap AV1451
t = tracers_names.index("A")

for d in days[0:4]:
    for s in sessions:
        a_24 = []
        a_24.extend([x[(t,d,sess)] for sess in sessions if sess >= s])#get rest of sessions in sameday
        a_24.extend([x[(t,d+1,sess)] for sess in sessions if sess < s])#get sessions within 24 hours later
        model.Add(sum(a_24) <= 1)

In [13]:
#12:00 earliest sess for AV1451
t = tracers_names.index("A")
a_early = []
for d in days:
    for s in sessions[0:8]:
        a_early.append(x[(t,d,s)])
        #print("x{}{}{}".format(d,s,t)) #sum() = 0
model.Add(sum(a_early) == 0)

In [14]:
#PIB 3 hr gap
t = tracers_names.index("P")
for d in days:
    for s in sessions[0:12]:
        p_gap = []
        for num in range(6):
            p_gap.append(x[(t,d,s+num)])
        model.Add(sum(p_day) <= 1)

In [15]:
#MK6240 only sessions 10 through 16; Tues-Fri
t = tracers_names.index("M")
m_days = []
sess_all = [0,1,2,3,4,5,6,7,8,16]
for d in days[1:5]:
    for s in list(sessions[0:9])+list(sessions[16:18]):
        m_days.append(x[(t,d,s)])
model.Add(sum(m_days) == 0)

In [16]:
#MK6240 only sessions 8 through 13; Mon
t = tracers_names.index("M")
d = day_names.index("Mon")
m_mon = []
for s in list(sessions[0:7])+list(sessions[13:18]):
    m_mon.append(x[(t,d,s)])
model.Add(sum(m_mon) == 0)

In [17]:
#UCB-J session 6 or 12 Tues-Fri
t = tracers_names.index("U")
for d in days[1:5]:
    u_sess = []
    for s in list(sessions[0:5])+list(sessions[6:11])+list(sessions[12:18]):
        u_sess.append(x[(t,d,s)])
    model.Add(sum(u_sess) == 0)

In [18]:
#UCB-J session 14 Mon
t = tracers_names.index("U")
d = day_names.index("Mon")
u_mon = []
for s in list(sessions[0:13])+list(sessions[14:18]):
    u_mon.append(x[(t,d,s)])
model.Add(sum(u_mon) == 0)

In [ ]:
#Incentivize having a PIB scan followed by an MK6240 in the same day
pib_mk_same_day = []
for d in days:
    #pib_mk = []
    for s in sessions:
        mk_sess = []
        for sess in range(s,len(sessions)):
            mk_sess.append(x[(tracers_names.index("M"),d,sess)])
        pib_mk_same_day.append([x[(tracers_names.index("P"), d, s)], mk_sess])
        #print(x[(tracers_names.index("P"), d, s)])
        #print(mk_sess)
    #print(pib_mk)
    #pib_mk_same_day.append(x[(tracers_names.index("P"), d, s)] * sum(mk_sess))

In [ ]:
sum(pib_mk_same_day[0][1])

In [ ]:
#Incentivize having 2 AV1451 in the same day
av_two = []
for d in days:
    av_same = []
    for s in sessions:
        

In [ ]:
#Incentivize having 2 MK6240 in the same day
mk_two = []

In [ ]:
#Incentivize having 3 MK6240 in the same day
mk_three = []

In [ ]:
# pylint: disable=g-complex-comprehension
model.Maximize( sum(4*pib_mk_same_day[s][0]*sum(pib_mk_same_day[s][1])
              for t in tracers for d in days for s in sessions))

#    sum(shift_requests[t][d][s] * x[(t, d, s)] for t in tracers
#        for d in days for s in sessions))

In [26]:
# pylint: disable=g-complex-comprehension
model.Maximize(sum(third[t][d][s] * x[(t, d, s)] for t in tracers for d in days for s in sessions))


In [30]:
solver = cp_model.CpSolver()
status = solver.Solve(model)


if status == cp_model.OPTIMAL:
    print('Solution:')
    for d in days:
        print('Day', d)
        for t in tracers:
            for s in sessions:
                if solver.Value(x[(t, d, s)]) == 1:
                    if third[t][d][s] == 1:
                        print('Nurse', t, 'works shift', s, '(requested).')
                    else:
                        print('Nurse', t, 'works shift', s,
                              '(not requested).')
        print()
#    print(f'Number of shift requests met = {solver.ObjectiveValue()}',
#          f'(out of {num_nurses * min_shifts_per_nurse})')
else:
    print('No optimal solution found !')

Solution:
Day 0
Nurse 0 works shift 0 (requested).
Nurse 0 works shift 4 (requested).
Nurse 0 works shift 16 (requested).
Nurse 2 works shift 8 (requested).
Nurse 2 works shift 12 (requested).

Day 1
Nurse 0 works shift 0 (requested).
Nurse 0 works shift 4 (requested).
Nurse 0 works shift 8 (requested).
Nurse 1 works shift 16 (requested).
Nurse 2 works shift 12 (requested).

Day 2
Nurse 0 works shift 0 (requested).
Nurse 0 works shift 4 (requested).
Nurse 0 works shift 8 (requested).
Nurse 1 works shift 16 (requested).
Nurse 2 works shift 12 (requested).

Day 3
Nurse 0 works shift 0 (requested).
Nurse 0 works shift 4 (requested).
Nurse 0 works shift 8 (requested).
Nurse 1 works shift 16 (requested).
Nurse 2 works shift 12 (requested).

Day 4
Nurse 0 works shift 0 (requested).
Nurse 2 works shift 11 (requested).
Nurse 2 works shift 15 (requested).
Nurse 3 works shift 5 (requested).



In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
sch_data = pd.DataFrame([
    dict(Session="08:00-10:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="08:30-10:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="09:00-11:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="09:30-11:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="10:00-12:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="10:30-12:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="11:00-13:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="11:30-13:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="12:00-14:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="12:30-14:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="13:00-15:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="13:30-15:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="14:00-16:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="14:30-16:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="15:00-17:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="15:30-17:30", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="16:00-18:00", Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session="08:00-10:00", Start=0, Finish=1, Tracer="PIB", color="2"),
    dict(Session="11:30-13:30", Start=0, Finish=1, Tracer="MK6240", color="3"),
    dict(Session="14:30-16:30", Start=0, Finish=1, Tracer="UCB-J", color="4"),
    dict(Session="12:00-14:00", Start=1, Finish=2, Tracer="AV1451", color="5")


])
sch_data['delta'] = sch_data['Finish'] - sch_data['Start']

#sch_data.append({'Session' : "08:00-10:00",
#                'Start' : 1,
#                'Finish' : 1,
#                'Tracer' : "PIB"}, ignore_index=True)

#labels_dict = {"y":"Mon"}
fig = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig.update_yaxes(autorange="reversed")
fig.layout.xaxis.type = 'linear'
for d in fig.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()
    
#fig.data[0].x = sch_data.delta.tolist()

fig.show()

In [ ]:
fig.to_dict()

In [ ]:
file_name = '~/Downloads/scheduling_graph'
fig.write_html(f"{file_name}.html")